In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier 
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import ParameterGrid, StratifiedKFold, RepeatedStratifiedKFold, train_test_split 
import matplotlib.pyplot as plt import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from mrmr import mrmr_classif
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVC as SVM
from sklearn.svm import SVC as SVC
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

%matplotlib inline
sns.set(rc={"figure.figsize": (10, 8), "font.size": 10}) sns.set_style("whitegrid")
sns.set_context("talk", font_scale=1.0, rc={"lines.linewidth": 2.5})


Load feature dataframes of three observers and select robust features under different observer annotations

In [ ]:
wilms_features = pd.read_csv ('/Users/koska/Downloads/wilms_features.csv') 
nb_features = pd.read_csv('/Users/koska/Downloads/nb_features.csv') 
wilms_features_obs2 = pd.read_csv('/Users/koska/Downloads/wilms_features_obs2.csv') 
nb_features_obs2 = pd.read_csv('/Users/koska/Downloads/nb_features_obs2.csv') 
wilms_featurs_obs3 = pd.read_csv('/Users/koska/Downloads/wilms_features_obs3.csv') 
nb_features_obs3 = pd.read_csv('/Users/koska/Downloads/nb_features_obs3.csv')

In [ ]:
inter_obs_nb_obs1 = nb_features [:18] 
inter_obs_nb_obs2 = nb_features_obs2 [:18] 
inter_obs_nb_obs3 = nb_features_obs3 [:18] 
nb_label = np.zeros ((18))

inter_obs_wilms_obs1 = wilms_features [:22] 
inter_obs_wilms_obs2 =wilms_features_obs2[:22] 
inter_obs_wilms_obs3 = wilms_featurs_obs3[:22] 
wilms_label = np.ones ((22))

obs1_df = pd.concat ((inter_obs_nb_obs1,inter_obs_wilms_obs1),axis = 0) 
obs2_df = pd.concat ((inter_obs_nb_obs2,inter_obs_wilms_obs2),axis = 0) 
obs3_df = pd.concat ((inter_obs_nb_obs3,inter_obs_wilms_obs3),axis = 0)


icc_obs1_obs2 = obs1_df.corrwith (other=obs2_df) 
robust_features_12 =icc_obs1_obs2 [icc_obs1_obs2 >=0.9]
icc_obs1_obs3 = obs1_df.corrwith (other=obs3_df) 
robust_features_13 =icc_obs1_obs3 [icc_obs1_obs3 >=0.9]
icc_obs2_obs3 = obs2_df.corrwith (other= obs3_df) 
robust_features_23 =icc_obs2_obs3 [icc_obs2_obs3 >=0.9]
robust_features_obs1_2= main_feature_df.loc[:,robust_features_12.index] 
robust_features_obs1_3 =main_feature_df.loc[:,robust_features_13.index] 
robust_features_obs2_3 =main_feature_df.loc[:,robust_features_23.index]

s1 = set(robust_features_obs1_2)
s2 = set(robust_features_obs1_3)
s3 = set(robust_features_obs2_3)
set12_13 = s1.intersection(s2)
set13_23 = s2.intersection(s3) 
result_set = set12_13 .intersection(s3) 
common12_13 = list (set12_13) 
common12_23 = list (set12_23) 
common13_23 = list (set13_23) 
final_list = list(result_set)


Then select non redundant and high variance features

In [ ]:
def select_non_redundan (corr_columns, robust_data,ext_data ):
    columns = np.full((corr_columns.shape[0],), True, dtype=bool) 
    for i in range(corr_columns.shape[0]):
        for j in range(i+1, corr_columns.shape[0]): 
            if abs (corr_columns.iloc[i,j]) >= 0.9:
                if columns[j]: 
                    columns[j] = False
    selected_columns_ = robust_data.columns[columns] 
    data_ =robust_data[selected_columns_] 
    ext_data_df = ext_data [selected_columns_] 
    return data_, ext_data_df

In [ ]:
hutf_complete_df = pd.read_csv ('/Users/koska/Documents/hutf_complete_df.csv') 
ext_complete_df = pd.read_csv ('/Users/koska/Documents/external_complete_df.csv')

hacettepe_klinik_last_analysis = pd.read_csv ('/Users/koska/Documents/hacettepe_klinik_last_analysis.csv') 
dm_klinik_last_analysis = pd.read_csv ('/Users/koska/Documents/dm_klinik_last_analysis.csv')

hutf_labels = hutf_complete_df ['label']
ext_labels = ext_complete_df ['label']

hütf_just_features = hutf_complete_df.drop(['label','Patient code'],axis = 1) 
ext_just_features = ext_complete_df.drop(['label','Patient code'],axis = 1)

hutf_robust_features = hütf_just_features [final_list]
ext_robust_features = ext_just_features [final_list]

hutf_nonredundan_features = hutf_robust_features.corr()
non_redundan_hütf_df, non_redundan_ext_df = select_non_redundan (hutf_nonredundan_features ,hutf_robust_features,
                                                                 ext_robust_features)
                                                                 

sel_hutf = VarianceThreshold(threshold=0.05)
selection_hutf = sel_hutf.fit_transform(non_redundan_hütf_df)
concol_hutf = [column for column in non_redundan_hütf_df.columns if column not in non_redundan_hütf_df.columns[sel_hutf.get_support()]] 
hutf_var_dropped = non_redundan_hütf_df.drop(concol_hutf,axis=1)
ext_var_droppedd = non_redundan_ext_df.drop (concol_hutf,axis =1)                                                                 

Now since we have robust, nonredundant and high variance features, we can proceed with supervised feature selection.

In [ ]:
def diff_express(X_p, X_n):
    n_p = len(X_p) 
    n_n = len(X_n)
    t, p = stats.ttest_ind(X_p, X_n, axis=0, equal_var=False)
    mu_p = X_p.mean(axis=0) 
    mu_n = X_n.mean(axis=0) 
    mean_diff = mu_p - mu_n 
    var_p = X_p.var(axis=0) 
    var_n = X_n.var(axis=0)
    S_pooled = np.sqrt( ((n_p-1)*var_p + (n_n-1)*var_n)/(n_p + n_n-2) ) 
    Cd = np.divide(mean_diff,S_pooled)
    thr = (mu_p * np.sqrt(var_n) + mu_n * np.sqrt(var_p)) / (np.sqrt(var_n) + np.sqrt(var_p)) 
    n_above = np.sum(X_n > thr, axis=0)
    p_below = np.sum(X_p < thr, axis=0)
    mc_rate = 0.5* ( n_above / n_n + p_below / n_p)
    return p, Cd, mc_rate


def volcano(dat, col_label='label'): 
    positive = dat[dat[col_label]==1].drop(col_label,axis=1).values 
    n_p = len(positive)
    negative = dat[dat[col_label]==0].drop(col_label, axis=1).values 
    n_n = len(negative)
    pval, cd, mcr = diff_express(positive, negative)
    arr = np.hstack((pval.reshape(-1,1),mcr.reshape(-1,1)))
    df_ = pd.DataFrame(arr, index=list(dat.columns)[:-1], columns=['pval','mc_rate'])
    return df_

First scale the feature set with standart scaler. Then apply volcano, mrmr, rfe and filter based feature selection.

In [ ]:
hutf_scaler = preprocessing.StandardScaler().fit(hutf_var_dropped) 
hutf_scaled = hutf_scaler.transform (hutf_var_dropped)
ext_scaled = hutf_scaler.transform (ext_var_droppedd)
hutf_scaled_df = pd.DataFrame (hutf_scaled, columns = hutf_var_dropped.columns ) 
hutf_scaled_df_with_labels = hutf_scaled_df.copy ()
hutf_scaled_df_with_labels ['label'] = hutf_labels
ext_scaled_df = pd.DataFrame (ext_scaled, columns = ext_var_droppedd.columns ) 
ext_scaled_df_with_labels = ext_scaled_df.copy ()
ext_scaled_df_with_labels ['label'] = ext_labels

In [ ]:
df_v_hutf = volcano(hutf_scaled_df_with_labels)

df_v_hutf['log10p'] = df_v_hutf ['pval'].apply(lambda x: -np.log10(x))
volcano_selected_hutf = df_v_hutf[(df_v_hutf['log10p'] > 4.) & (df_v_hutf['mc_rate'] < 0.3)]
volcano_selected_columns_hutf = list (volcano_selected_hutf.index)
X_volcano_features_hutf = hutf_scaled_df [volcano_selected_columns_hutf] 
X_volcano_features_hutf ['label'] = np.array (hutf_labels) 
X_volcano_features_hutf.to_csv ('X_volcano_features_hutf.csv')

X_volcano_features_ext = ext_scaled_df [volcano_selected_columns_hutf] 
X_volcano_features_ext ['label'] = np.array (ext_labels) 
X_volcano_features_ext.to_csv ('X_volcano_features_ext.csv')

fig = plt.figure(figsize=(8,6))
sns.scatterplot(data=df_v_hutf, x='mc_rate', y='log10p', alpha=0.8) 
plt.xlabel('Misclassification Rate')
plt.ylabel('log10(pval)')
plt.show()

In [ ]:
mrmr_selected_features_hutf = mrmr_classif(hutf_scaled_df, hutf_labels, K = 6) 
X_mrmr_features_hutf = hutf_scaled_df[mrmr_selected_features_hutf] 
X_mrmr_features_ext = ext_scaled_df[mrmr_selected_features_hutf] 
X_mrmr_features_hutf ['label'] = np.array (hutf_labels) 
X_mrmr_features_ext ['label'] = np.array (ext_labels) 
X_mrmr_features_hutf.to_csv ('X_mrmr_features_hutf.csv')
X_mrmr_features_ext.to_csv ('X_mrmr_features_ext.csv')

In [ ]:
model = LogisticRegression()
rfe_hutf = RFE(model,n_features_to_select=6)
fit_hutf = rfe_hutf.fit(hutf_scaled_df, hutf_labels) 
rfe_selected_hutf = hutf_scaled_df.columns[fit_hutf.support_] 
X_rfe_features_hutf = hutf_scaled_df[rfe_selected_hutf] 
X_rfe_features_ext = ext_scaled_df[rfe_selected_hutf] 
X_rfe_features_hutf ['label'] = np.array (hutf_labels) 
X_rfe_features_ext ['label'] = np.array (ext_labels) 
X_rfe_features_hutf.to_csv ('X_rfe_features_hutf.csv') 
X_rfe_features_ext.to_csv ('X_rfe_features_ext.csv')

In [ ]:
def get_filter_best_features (all_features, labels):
    wrap_feat_select = ExtraTreesClassifier ()
    wrap_feat_select.fit(all_features, labels)
    train_pd = pd.DataFrame (all_features)
    dfcolumns = pd.DataFrame (train_pd.columns)
    feat_importances = pd.Series (wrap_feat_select.feature_importances_, index = dfcolumns) feat_importances = feat_importances.nlargest (6)
    return feat_importances
def get_filter_reduced (feat_importances,all_features): 
    wrapper_index= feat_importances.index 
    reduced_wrapper= all_features [:,wrapper_index] 
    return reduced_wrapper
def plot_feat_importances (feat_importances): 
    feat_importances.plot (kind = 'barh') 
    plt.show ()

In [ ]:
filter_features_hutf = get_filter_best_features (hutf_scaled_df, hutf_labels) 
filter_features_hutf_ = list (filter_features_hutf.index)
filter_features_= [ filter_features_hutf_[i][0] for i in range(len (filter_features_hutf_)) ] 
X_filter_features_hutf = hutf_scaled_df [filter_features_]
X_filter_features_ext = ext_scaled_df [filter_features_] 
X_filter_features_hutf ['label'] = np.array (hutf_labels) 
X_filter_features_ext ['label'] = np.array (ext_labels) 
X_filter_features_hutf.to_csv ('X_filter_features_hutf.csv') 
X_filter_features_ext.to_csv ('X_filter_features_ext.csv')

Since we completed feature selection, we can train our models.

In [ ]:
def merge_four (dict1,dict2,dict3,dict4):
    res = {**dict1, **dict2, **dict3, **dict4} 
    return res

def get_svm_scores_cv (df):
    labels,features = set_features_labels (df)
    scoring = ('f1', 'recall', 'precision', 'accuracy', 'roc_auc')
    cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=10, random_state=1)
    svm = SVM(max_iter=10000)
    svm_over = SVM(max_iter=10000)
    clf_rbf = SVC(kernel="rbf",C=5,max_iter=10000)
    clf_weight = SVC(kernel="linear", max_iter=10000,class_weight='balanced')
    oversample = SMOTE()
    over_X, over_y = oversample.fit_resample(features,labels)
    scores_svm = cross_validate(svm, features, labels, scoring=scoring, cv=cv) scores_svm_over = cross_validate(svm_over, over_X, over_y, scoring=scoring, cv=cv) scores_svm_rbf = cross_validate(clf_rbf, features, labels, scoring=scoring, cv=cv) scores_svm_weight = cross_validate(clf_weight, features, labels, scoring=scoring, cv=cv)
    svm_dict = build_scores_dict (scores_svm, 'svm')
    svm_over_dict = build_scores_dict (scores_svm_over, 'over')
    svm_rbf_dict = build_scores_dict (scores_svm_rbf, 'rbf')
    svm_weight_dict = build_scores_dict (scores_svm_weight, 'weighted') result_dict = merge_four (svm_dict,svm_over_dict,svm_rbf_dict,svm_weight_dict)
    #print_cv_scores (scores_svm,'svm')
    #print_cv_scores (scores_svm_over, 'over')
    #print_cv_scores (scores_svm_rbf, 'rbf')
    #print_cv_scores (scores_svm_weight, 'weighted')
    return result_dict

def get_rf_scores_cv (df):
    labels,features = set_features_labels (df)
    scoring = ('f1', 'recall', 'precision', 'accuracy', 'roc_auc')
    cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=10, random_state=1)
    rf_model = RandomForestClassifier(n_estimators=150, random_state= 10)
    rf_over = RandomForestClassifier(n_estimators=150, random_state= 10) balanced_rf_model = BalancedRandomForestClassifier(n_estimators=150, random_state=2) dec_tree_model = DecisionTreeClassifier()
    oversample = SMOTE()
    over_X, over_y = oversample.fit_resample(features,labels)
    scores_rf = cross_validate(rf_model, features, labels, scoring=scoring, cv=cv)
    scores_rf_over = cross_validate( rf_over, over_X, over_y, scoring=scoring, cv=cv) scores_rf_balanced = cross_validate( balanced_rf_model, features, labels, scoring=scoring, cv=cv) scores_dec_tree = cross_validate(dec_tree_model, features, labels, scoring=scoring, cv=cv)
    rf_dict = build_scores_dict (scores_rf, 'RF')
    rf_over_dict = build_scores_dict (scores_rf_over, 'over')
    rf_balanced_dict = build_scores_dict (scores_rf_balanced, 'balanced') dec_tree_dict = build_scores_dict (scores_dec_tree, 'dec_tree')
    result_dict = merge_four (rf_dict,rf_over_dict,rf_balanced_dict,dec_tree_dict)
    #print_cv_scores (scores_rf,'RF') #print_cv_scores (scores_rf_over, 'over') #print_cv_scores (scores_rf_balanced, 'balanced') #print_cv_scores (scores_dec_tree, 'DT')
    return result_dict

def set_features_labels (df):
    labels = df ['label']
    features = df.drop ('label', axis =1) 
    if 'Unnamed: 0' in features.keys():
        features = features.drop ('Unnamed: 0', axis=1) 
    else:
        features = features 
    return labels, features

def print_scores (labels, preds,pref):
    print(str(pref) + 'Recall: %.3f' % recall_score(labels, preds)) 
    print(str(pref) +'Accuracy: %.3f' % accuracy_score(labels, preds)) 
    print(str(pref) +'F1 Score: %.3f' % f1_score(labels, preds)) 
    print(str(pref) +'Precision: %.3f' % precision_score(labels, preds)) 
    print(str(pref) +'ROC_AUC: %.3f' % roc_auc_score(labels, preds))

def print_cv_scores (scores,pref):
    print(str(pref) + 'Mean f1: %.3f' % mean(scores['test_f1'])) 
    print(str(pref) + 'Std f1: %.3f' % stdev(scores['test_f1'])) 
    print(str(pref) + 'Mean recall: %.3f' % mean(scores['test_recall'])) 
    print(str(pref) + 'Std recall: %.3f' % stdev(scores['test_recall'])) 
    print(str(pref) + 'Mean precision: %.3f' % mean(scores['test_precision'])) 
    print(str(pref) + 'Std precision: %.3f' % stdev(scores['test_precision'])) 
    print(str(pref) + 'Mean accuracy: %.3f' % mean(scores['test_accuracy'])) 
    print(str(pref) + 'Std accuracy: %.3f' % stdev(scores['test_accuracy'])) 
    print(str(pref) + 'Mean roc_auc: %.3f' % mean(scores['test_roc_auc'])) 
    print(str(pref) + 'Std roc_auc: %.3f' % stdev(scores['test_roc_auc']))

def build_scores_dict (scores,pref):
    scores_dict= {str(pref) + '_Mean f1:':mean(scores['test_f1']),
    str(pref) + '_Std f1:': stdev(scores['test_f1']),
    str(pref) + '_Mean recall:': mean(scores['test_recall']), 
    str(pref) + '_Std recall:': stdev(scores['test_recall']), 
    str(pref) + '_Mean precision:': mean(scores['test_precision']), 
    str(pref) + '_Std precision:': stdev(scores['test_precision']), 
    str(pref) + '_Mean accuracy:': mean(scores['test_accuracy']), 
    str(pref) + '_Std accuracy:': stdev(scores['test_accuracy']), 
    str(pref) + '_Mean roc_auc:': mean(scores['test_roc_auc']), 
    str(pref) + '_Std roc_auc:': mean(scores['test_roc_auc'])}
    return scores_dict

def get_svm_scores (df):
    labels,features = set_features_labels (df)
    svm = SVM(max_iter=10000)
    svm_over = SVM(max_iter=10000)
    #clf = svm.SVC(kernel="linear", C=1.0)
    clf_rbf = SVC(kernel="rbf",C=5,max_iter=10000)
    clf_weight = SVC(kernel="linear", max_iter=10000,class_weight='balanced')
    oversample = SMOTE()
    over_X, over_y = oversample.fit_resample(features,labels)
    otr_features, ote_features, otr_labels, ote_labels = train_test_split(over_X, over_y, test_size=0.1, stratify=over_y)
    tr_features, te_features, tr_labels, te_labels = train_test_split(features, labels, test_size=0.1, stratify=labels)
    svm.fit(tr_features, tr_labels) svm_over.fit (otr_features, otr_labels) 
    clf_rbf.fit (tr_features, tr_labels) 
    #clf.fit(X, y) clf_weight.fit(tr_features, tr_labels)
    y_pred_svm = svm.predict(te_features) 
    y_pred_clf_rbf = clf_rbf.predict(te_features) 
    y_pred_weight = clf_weight.predict(te_features) 
    y_pred_over = svm_over.predict (ote_features)
    #print_scores (te_labels,y_pred_svm,'svm')
    #print_scores (te_labels,y_pred_clf_rbf, 'rbf')
    #print_scores (te_labels,y_pred_weight, 'weighted')
    #print_scores (ote_labels, y_pred_over, 'over')
    return y_pred_svm,y_pred_clf_rbf,y_pred_weight,y_pred_over, svm,clf_rbf, clf_weight,svm_over

def get_rf_scores (df):
    labels,features = set_features_labels (df)
    rf_model = RandomForestClassifier(n_estimators=150, random_state= 10)
    rf_over = RandomForestClassifier(n_estimators=150, random_state= 10) balanced_rf_model = BalancedRandomForestClassifier(n_estimators=150, random_state=2) dec_tree_model = DecisionTreeClassifier()
    oversample = SMOTE()
    over_X, over_y = oversample.fit_resample(features,labels)
    rf_model.fit(features, labels)
    balanced_rf_model.fit (features, labels)
    rf_over.fit (over_X, over_y)
    dec_tree_model.fit(features, labels)
    y_pred_rf = rf_model.predict(features) 
    y_pred_balanced_rf = balanced_rf_model.predict(features) 
    y_pred_dec_tree = dec_tree_model.predict(features) 
    y_pred_over = rf_over.predict (over_X)
    #print_scores (labels,y_pred_rf,'RandomForest') 
    #print_scores (labels,y_pred_balanced_rf, 'BRF') 
    #print_scores (labels,y_pred_dec_tree, 'DecTree') 
    #print_scores (over_y, y_pred_over, 'Over')
    return y_pred_rf,y_pred_balanced_rf,y_pred_dec_tree,y_pred_over,rf_model,balanced_rf_model, dec_tree_model, rf_over

In [ ]:
cv_svm_hutf_filter =get_svm_scores_cv (X_filter_features_hutf) 
cv_svm_hutf_mrmr =get_svm_scores_cv (X_mrmr_features_hutf) 
cv_svm__hutf_rfe =get_svm_scores_cv (X_rfe_features_hutf) 
cv_svm_hutf_volcano =get_svm_scores_cv (X_volcano_features_hutf)

cv_rf_hutf_filter =get_rf_scores_cv (X_filter_features_hutf) 
cv_rf_hutf_mrmr =get_rf_scores_cv (X_mrmr_features_hutf) 
cv_rf_hutf_rfe =get_rf_scores_cv (X_rfe_features_hutf) 
cv_rf_hutf_volcano =get_rf_scores_cv (X_volcano_features_hutf)

Use best model and best feature selection strategy to test with external validation set

In [ ]:
external_valid_scores = get_svm_scores (X_rfe_features_ext)

Now let's combine all the best radiomics features selected by 4 different supervised feature selection methods and clinical features. Then reapply secondary feature selection to find best of best subset.

In [ ]:
union_list = [volcano_selected_columns_hutf,mrmr_selected_features_hutf, X_rfe_features_hutf, X_filter_features_hutf] 
feature_union = set ().union (*union_list)

hutf_temp = hutf_scaled_df.copy ()
hutf_temp ['label'] = np.array (hutf_labels) 
union_df = hutf_temp[feature_union]

hutf_clinic_temp = hacettepe_klinik_last_analysis.copy () 
hutf_clinic_temp = hutf_clinic_temp.drop (['label'],axis = 1) 
hutf_clinic_temp = hutf_clinic_temp.reset_index(drop=True)
clinico_rad_hutf_raw = pd.concat ((hutf_clinic_temp,union_df),axis = 1)

In [ ]:
model = LogisticRegression()
rfe_clinico_rad = RFE(model,n_features_to_select=6)
fit_clinico_rad = rfe_clinico_rad.fit(clinico_rad_hutf_raw, np.array (hutf_labels)) 
rfe_selected_clinico_rad = clinico_rad_hutf_raw.columns[fit_clinico_rad.support_] 
X_rfe_features_clinico_rad = clinico_rad_hutf_raw[rfe_selected_clinico_rad] 
X_rfe_features_clinico_rad ['label'] = np.array (hutf_labels)

In [ ]:
ext_temp = ext_scaled_df.copy ()
ext_temp ['label'] = np.array (ext_labels) 
union_ext_df = ext_temp[feature_union]
ext_clinic_temp = dm_klinik_last_analysis.copy ()
dm_clinic_temp = ext_clinic_temp.drop (['label'],axis = 1) 
dm_clinic_temp = dm_clinic_temp.reset_index(drop=True)
clinico_rad_ext_raw = pd.concat ((dm_clinic_temp,union_ext_df),axis = 1) 
X_rfe_features_clinico_rad_ext = clinico_rad_ext_raw[rfe_selected_clinico_rad] 
X_rfe_features_clinico_rad_ext['label'] = np.array (ext_labels)

First use cross validation to explore performance estimates with the train set, then apply best performing model and features on the external test set

In [ ]:
cv_svm_clin_rad =get_svm_scores_cv (X_rfe_features_clinico_rad) 
cv_rf_clin_rad =get_rf_scores_cv (X_rfe_features_clinico_rad)

In [ ]:
external_valid_scores_clinico_radiomic = get_svm_scores (X_rfe_features_clinico_rad_ext)